In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.3-70b-versatile",api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000276FCFA20B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000276FD0DDAB0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage,SystemMessage

model.invoke(
    [HumanMessage(content="Hi, I am Avinash and i am a upcoming software engineer")]
)

AIMessage(content="Hello Avinash, it's nice to meet you. Congratulations on pursuing a career in software engineering. That's an exciting and in-demand field. What areas of software engineering interest you the most? Are you focusing on any specific programming languages, technologies, or industries? I'm here to help and chat about any topics related to software engineering.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 49, 'total_tokens': 119, 'completion_time': 0.254545455, 'prompt_time': 0.007662399, 'queue_time': 0.05016323, 'total_time': 0.262207854}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None}, id='run--152e22a4-6f48-42a4-8369-4567c8bd9d6a-0', usage_metadata={'input_tokens': 49, 'output_tokens': 70, 'total_tokens': 119})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, I am Avinash and i am a upcoming software engineer"),
    AIMessage(content="Hello Avinash, it's nice to meet you. Congratulations on pursuing a career in software engineering. That's an exciting and in-demand field. What areas of software engineering interest you the most? Are you focusing on any specific programming languages, technologies, or industries? I'm here to help and chat about any topics related to software engineering."),
    HumanMessage("Hey! Tell me what's my name and what do i do?")
])

AIMessage(content="Your name is Avinash, and you're an upcoming software engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 142, 'total_tokens': 158, 'completion_time': 0.058181818, 'prompt_time': 0.015647242, 'queue_time': 0.050910437999999995, 'total_time': 0.07382906}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run--28723145-19fc-4981-b06c-d87b957b7305-0', usage_metadata={'input_tokens': 142, 'output_tokens': 16, 'total_tokens': 158})

### Message History
We can use message history class to wrap out model and make it stateful. This will keep track of input and output of model, and store them in a datastore

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [9]:
config = { "configurable":{"session_id":"chat1"}}

In [11]:
response = with_message_history.invoke([
    HumanMessage(content="Hi my name is Avinash and i am a full stack web developer trying to learn generative AI")
], config = config)

response.content

"Nice to meet you, Avinash. As a full-stack web developer, you already have a solid foundation in programming and web development. Learning generative AI can be a fascinating and rewarding experience, and it's great that you're interested in exploring this field.\n\nTo get started, let's break down the key concepts and technologies involved in generative AI:\n\n1. **Deep Learning**: Generative AI relies heavily on deep learning techniques, such as neural networks and convolutional neural networks (CNNs).\n2. **Python Libraries**: You'll need to familiarize yourself with popular Python libraries like TensorFlow, PyTorch, or Keras, which provide tools and frameworks for building and training generative models.\n3. **Generative Models**: Study the different types of generative models, including:\n\t* Generative Adversarial Networks (GANs)\n\t* Variational Autoencoders (VAEs)\n\t* Autoregressive Models\n\t* Transformers\n4. **Mathematics**: Linear algebra, calculus, and probability theory 

In [17]:
response = with_message_history.invoke([
    HumanMessage("What is my name?"),
    
],config=config)

response.content

'Your name is Avinash.'

In [18]:
# Let's change the config --> change in session_id
config1 = { "configurable":{"session_id":"chat2"}}

response = with_message_history.invoke([
    HumanMessage("What is my name?"),
    
],config=config1)

response.content

"I don't know your name. I'm a large language model, I don't have the ability to access personal information about you, including your name. I can only respond based on the text you provide to me. If you'd like to share your name, I'd be happy to chat with you and use it in our conversation."

In [20]:
# Prompt templates
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI assistant whose main task is to explain the asked topic in simple words"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

In [22]:
chain.invoke({"messages":[HumanMessage("Tell me about database indexing")]})

AIMessage(content="**What is Database Indexing?**\n\nImagine you have a huge phonebook with millions of names and phone numbers. If you want to find a specific person's phone number, you would have to look through the entire book, which would take a long time.\n\nDatabase indexing is like creating a special table of contents for your phonebook. It helps the database find specific data quickly, without having to look through the entire database.\n\n**How Does Indexing Work?**\n\nWhen you create an index on a column in your database table, the database creates a separate data structure that contains the values from that column, along with a pointer to the location of the corresponding data in the table.\n\nThink of it like a map that shows the database where to find specific data. When you query the database, it can use the index to quickly locate the data, rather than having to scan the entire table.\n\n**Benefits of Indexing**\n\n1. **Faster Query Performance**: Indexing can significan

In [23]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [24]:
config3 = { "configurable":{"session_id":"chat3"}}

In [25]:
with_message_history.invoke(
    [
        HumanMessage("Hi my name is avinash")
    ],
    config=config3
)

AIMessage(content="Hello Avinash, it's nice to meet you. Is there something you'd like to know or learn about? I'm here to help and explain things in simple words. What's on your mind?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 60, 'total_tokens': 103, 'completion_time': 0.156363636, 'prompt_time': 0.004279283, 'queue_time': 0.051342327, 'total_time': 0.160642919}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None}, id='run--2da5d534-9f7e-40f7-a85d-4229e24dd52a-0', usage_metadata={'input_tokens': 60, 'output_tokens': 43, 'total_tokens': 103})

In [26]:
# Adding more complexity to prompt

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI assistant whose main task is to explain the asked topic in simple words. Explain the topics in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

In [27]:
res =  chain.invoke({
    "messages":[HumanMessage("Hi my name is avinash")],
    "language":"hindi"
})

In [28]:
res.content

'नमस्ते अविनाश, मैं आपका सहायक हूँ। मैं आपके प्रश्नों का उत्तर देने और आपकी जानकारी बढ़ाने में मदद करने के लिए तैयार हूँ। आज हम किस विषय पर चर्चा करना चाहते हैं?'

In [29]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [34]:
config4 = {"configurable":{"session_id":"chat4"}}

ans = with_message_history.invoke(
    {"messages":[
        HumanMessage("Hey I am avinash and i am a aspiring software developer"),
    ],
    "language":"hindi"},
    config=config4
)

In [35]:
ans

AIMessage(content='नमस्ते अविनाश, मैं आपका सहायक हूँ। मैं आपको सॉफ्टवेयर विकास से संबंधित विषयों के बारे में सरल शब्दों में समझाने में मदद करूँगा।\n\nआप एक सॉफ्टवेयर डेवलपर बनने की आकांक्षा रखते हैं, यह बहुत अच्छा है! सॉफ्टवेयर विकास एक रोमांचक और तेजी से बढ़ता हुआ क्षेत्र है, जहाँ आप अपने विचारों और रचनात्मकता को वास्तविकता में बदल सकते हैं।\n\nक्या आप सॉफ्टवेयर विकास के किसी विशेष क्षेत्र में रुचि रखते हैं, जैसे कि वेब डेवलपमेंट, मोबाइल ऐप डेवलपमेंट, या कोई अन्य? या आप सीखना चाहते हैं कि सॉफ्टवेयर विकास की दुनिया में शुरुआत कैसे करें? मैं आपकी मदद के लिए तैयार हूँ!', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 277, 'prompt_tokens': 72, 'total_tokens': 349, 'completion_time': 1.007272727, 'prompt_time': 0.003667804, 'queue_time': 0.050787677, 'total_time': 1.010940531}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'finish_reason': 'stop', 'logprobs': None}, id='run--e659283a-c5bd-47c3-be5e-76949b37b1c8-0', usage_meta

In [42]:
ans = with_message_history.invoke(
    {"messages":[
        HumanMessage("What do you know about me?"),
    ],
    "language":"english"},
    config=config4
)

In [43]:
ans.content

"I know the following about you:\n\n* Your name is Avinash\n* You are an aspiring software developer\n* You initiated a conversation with me to discuss topics related to software development\n\nThat's all I know about you so far. If you'd like to share more about yourself, such as your interests, goals, or experiences, I'm here to listen and help if needed. What would you like to share about yourself?"

### Managing the conversation history
The list of messages can grow unbounded and can potentially overflow the llm context window, thats why it is needed to be managed.
An extra step is needed that limits the size of message we are passing in

In [44]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=70,
    strategy = "last",
    token_counter=model,
    include_system = True,
    allow_partial = False,
    start_on="human"
)